In [1]:
import torch
import pandas as pd
from scapy.all import sniff
from config.constants import FEATURES, CATEGORIES, BINARY_CATEGORIES
from features.pkt_to_features import update_flow_state
import time
import threading
from collections import Counter
import numpy as np
import sys

In [2]:
device = torch.device("cpu")
print(device)

cpu


In [3]:
multy_model_path = './models/tabnet_multy_100_traced.pt'
binary_model_path = './models/tabnet_binary_100_traced.pt'

In [4]:
multy_model = torch.jit.load(multy_model_path, map_location=device)
multy_model.eval()

binary_model = torch.jit.load(binary_model_path, map_location=device)
binary_model.eval()

RecursiveScriptModule(
  original_name=TabNet
  (initial_bn): RecursiveScriptModule(original_name=BatchNorm1d)
  (initial_splitter): RecursiveScriptModule(
    original_name=FeatureTransformer
    (shared): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Linear)
        (1): RecursiveScriptModule(original_name=BatchNorm1d)
        (2): RecursiveScriptModule(original_name=GLU)
        (3): RecursiveScriptModule(original_name=Dropout)
      )
      (1): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=Linear)
        (1): RecursiveScriptModule(original_name=BatchNorm1d)
        (2): RecursiveScriptModule(original_name=GLU)
        (3): RecursiveScriptModule(original_name=Dropout)
      )
    )
    (independent): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        origin

In [5]:
flow_features_buffer = []
last_prediction_time = time.time()
PREDICTION_INTERVAL = 5

In [6]:
def prepare_input_for_model(features_list, expected_features):
    if not features_list:
        return None
    print(f"Длина features_list: {len(features_list)}")

    df = pd.DataFrame(features_list)
    print(f"Созданный DataFrame: {df.shape}")
    for feat in expected_features:
        if feat not in df.columns:
            df[feat] = 0
    df = df[expected_features]
    print(f"DataFrame после заполнения: {df.shape}")
    return torch.FloatTensor(df.values).to(device)

In [7]:
def predict_traffic_type():
    global last_prediction_time, flow_features_buffer
    # Подготовка признаков
    input_tensor = prepare_input_for_model(flow_features_buffer, FEATURES)
    if input_tensor is None:
        print("Ошибка подготовки входного тензора")
        return

    # Предсказание с бинарной моделью
    with torch.no_grad():
        binary_output, _, _ = binary_model(input_tensor)
        binary_probs = torch.softmax(binary_output, dim=1)  # Вероятности для бинарной классификации
        binary_preds = binary_output.argmax(dim=1).cpu().numpy()  # Предсказания для всех строк

    # Проверка на наличие атак (индекс 0)
    if 0 in binary_preds:
        print(f"[{time.strftime('%H:%M:%S')}] Обнаружена атака! Программа остановлена.")
        return

    # Если все предсказания нормальные (индекс 1), продолжаем с многоклассовой моделью
    with torch.no_grad():
        multy_output, _, _ = multy_model(input_tensor)
        multy_probs = torch.softmax(multy_output, dim=1)  # Вероятности для каждого класса
        multy_preds = multy_output.argmax(dim=1).cpu().numpy()  # Предсказания для всех строк
        max_probs = multy_probs.max(dim=1)[0].cpu().numpy()  # Максимальные вероятности для каждой строки

    # Определение доминирующего класса
    dominant_class = Counter(multy_preds).most_common(1)[0][0]
    avg_max_prob = np.mean(max_probs)  # Средняя максимальная вероятность

    # Если средняя вероятность < 0.5, метка "unknown"
    if avg_max_prob < 0.5:
        dominant_label = "unknown"
    else:
        dominant_label = CATEGORIES[dominant_class]

    print(f"[{time.strftime('%H:%M:%S')}] Тип трафика: {dominant_label} (средняя вероятность: {avg_max_prob:.4f})")

    # Очистка буфера
    flow_features_buffer = []
    last_prediction_time = time.time()

In [8]:
def schedule_prediction():
    global last_prediction_time, flow_features_buffer
    current_time = time.time()
    elapsed_time = current_time - last_prediction_time

    if elapsed_time >= PREDICTION_INTERVAL:
        predict_traffic_type()
        last_prediction_time = current_time

    # Запланировать следующую проверку
    threading.Timer(1.0, schedule_prediction).start()

In [9]:
def packet_handler(pkt):
    """
    Обработчик пакетов для scapy.
    """
    features = update_flow_state(pkt)
    if features:
        # Сохраняем признаки в буфер для периодического предсказания
        flow_features_buffer.append(features)

In [10]:
threading.Timer(1.0, schedule_prediction).start()
# Запуск перехвата пакетов
sniff(prn=packet_handler, store=0)

Длина features_list: 94
Созданный DataFrame: (94, 78)
DataFrame после заполнения: (94, 77)
[00:14:14] Обнаружена атака! Программа остановлена.
Длина features_list: 279
Созданный DataFrame: (279, 78)
DataFrame после заполнения: (279, 77)
[00:14:19] Обнаружена атака! Программа остановлена.
Длина features_list: 674
Созданный DataFrame: (674, 78)
DataFrame после заполнения: (674, 77)


Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1433, in run
    self.function(*self.args, **self.kwargs)
  File "C:\Users\owtf0\AppData\Local\Temp\ipykernel_16076\3313170923.py", line 7, in schedule_prediction
  File "C:\Users\owtf0\AppData\Local\Temp\ipykernel_16076\3997884772.py", line 11, in predict_traffic_type
  File "E:\JetBrains Projects\PycharmProjects\network-classificator\.venv\Lib\site-packages\torch\nn\modules\module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "E:\JetBrains Projects\PycharmProjects\network-classificator\.venv\Lib\site-packages\torch\nn\modules\module.py", li

<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>